# Using Opik with Groq

Opik integrates with Groq to provide a simple way to log traces for all Groq LLM calls. This works for all Groq models.

## Creating an account on Comet.com

[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) and grab you API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik&utm_medium=colab&utm_content=openai&utm_campaign=opik) for more information.

In [1]:
%pip install --upgrade opik litellm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.9/217.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [2]:
import opik

opik.configure(use_local=False)

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "dasarpai" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


## Preparing our environment

First, we will set up our OpenAI API keys.

In [3]:
import os
import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


## Configure LiteLLM

Add the LiteLLM OpikTracker to log traces and steps to Opik:

In [4]:
import litellm
import os
from litellm.integrations.opik.opik import OpikLogger
from opik import track
from opik.opik_context import get_current_span_data

os.environ["OPIK_PROJECT_NAME"] = "grok-integration-demo"
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

## Logging traces

Now each completion will logs a separate trace to LiteLLM:

In [5]:
prompt = """
Write a short two sentence story about Opik.
"""

response = litellm.completion(
    model="groq/llama3-8b-8192",
    messages=[{"role": "user", "content": prompt}],
)

print(response.choices[0].message.content)

I'm happy to help! However, I must inform you that Opik is likely a fictional character or not a well-known historical figure, so I'll create a short story about a character with that name. Here it is:

Opik, a skilled mechanic from a small town, accidentally discovered a rare, ancient artifact while working on an old motorcycle, which changed the course of his life forever. With newfound purpose, Opik used his discovery to travel the world, uncovering hidden treasures and solving mysteries that had been shrouded in mystery for centuries.


The prompt and response messages are automatically logged to Opik and can be viewed in the UI.

![Groq Cookbook](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/cookbook/groq_trace_cookbook.png)

In [7]:
f"{response.choices[0].message.content}"

"I'm happy to help! However, I must inform you that Opik is likely a fictional character or not a well-known historical figure, so I'll create a short story about a character with that name. Here it is:\n\nOpik, a skilled mechanic from a small town, accidentally discovered a rare, ancient artifact while working on an old motorcycle, which changed the course of his life forever. With newfound purpose, Opik used his discovery to travel the world, uncovering hidden treasures and solving mysteries that had been shrouded in mystery for centuries."

In [10]:
import IPython

IPython.display.Markdown(f"{response.choices[0].message.content}")

I'm happy to help! However, I must inform you that Opik is likely a fictional character or not a well-known historical figure, so I'll create a short story about a character with that name. Here it is:

Opik, a skilled mechanic from a small town, accidentally discovered a rare, ancient artifact while working on an old motorcycle, which changed the course of his life forever. With newfound purpose, Opik used his discovery to travel the world, uncovering hidden treasures and solving mysteries that had been shrouded in mystery for centuries.

## Using it with the `track` decorator

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If Groq is called within one of these steps, the LLM call with be associated with that corresponding step:

In [8]:
@track
def generate_story(prompt):
    response = litellm.completion(
        model="groq/llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        metadata={
            "opik": {
                "current_span_data": get_current_span_data(),
            },
        },
    )
    return response.choices[0].message.content


@track
def generate_topic():
    prompt = "Generate a topic for a story about Opik."
    response = litellm.completion(
        model="groq/llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        metadata={
            "opik": {
                "current_span_data": get_current_span_data(),
            },
        },
    )
    return response.choices[0].message.content


@track
def generate_opik_story():
    topic = generate_topic()
    story = generate_story(topic)
    return story


generate_opik_story()

OPIK: Started logging traces to the "grok-integration-demo" project at https://www.comet.com/opik/dasarpai/projects/.


"What a fantastic idea! I'm excited to dive into the world of Dr. Opik and the Lost Expedition of Opik's Reach. Here's a possible story outline:\n\n**Act I: The Journey Begins**\n\n* Introduce Dr. Opik, a brilliant linguist and explorer, and his team of experts: a cryptologist, an archaeologist, a botanist, and a local guide.\n* Show the team's excitement and anticipation as they embark on their perilous journey to reach the ancient city.\n* Introduce the team's goal: to decipher the mysterious languages and symbols etched onto the city's architecture, which Dr. Opik believes will reveal the secrets of the civilization that once thrived there.\n\n**Act II: The City Unveiled**\n\n* The team arrives at the city, and Opik leads them on a guided tour of the crumbling structures.\n* As they explore, they begin to uncover clues that suggest the city was abandoned suddenly, leaving behind a great many secrets and mysteries.\n* Introduce the city's eerie, foreboding atmosphere, which starts to

In [11]:
import IPython

IPython.display.Markdown(f"{generate_opik_story()}")

I love the idea! "The Lost Expedition of Opik" has all the makings of a thrilling adventure story. The setting is unique and rich in history, and Opik's character has a great deal of potential for development. Here are some suggestions to further flesh out the plot and characters:

**Opik's Backstory:**

Give Opik a compelling backstory to explain his motivations and drive. Perhaps he's a seasoned explorer who's made a name for himself in Europe, but there's a personal tragedy or a sense of professional dissatisfaction that pushes him to seek out new lands and discoveries. This could be hinted at through his letters and journals, which serve as a narrative device to recount his journey.

**The Expedition Team:**

Assemble a diverse and dynamic team to accompany Opik on his expedition. This could include experts in various fields, locals who know the region, and possibly even a few rival explorers or treasure hunters. Each member could have their own secrets, motivations, and conflicts, which would create interesting dynamics and tension within the team.

**The Historical Context:**

Make sure to incorporate accurate historical details about the Silk Road, the cities Opik visits, and the cultures he encounters. This will add authenticity and depth to the story. Consider highlighting the complex politics and trade relationships between the various empires and city-states of the region, as well as the growing tensions between East and West.

**The Mysterious Civilizations:**

Create a sense of mystery and intrigue by slowly revealing the secrets of the ancient civilizations Opik discovers. This could involve cryptic clues, ancient texts, and oral traditions passed down through local communities. As Opik learns more, he'll need to navigate the complex web of alliances, rivalries, and treachery between the various powers vying for control of the region.

**The Antagonists:**

Introduce a range of antagonists, from rival explorers and treasure hunters to corrupt officials and power-hungry warlords. Consider making some of these characters complex and nuanced, with their own motivations and backstories. This will make the conflict more believable and challenging for Opik to overcome.

**The Climax:**

As Opik approaches the heart of the mystery, build up the tension and stakes. He could face a series of challenges and setbacks, such as treacherous terrain, violent confrontations, and treacherous betrayals. The climax could involve a dramatic showdown with a powerful adversary, a revelation about the ancient civilization, or a moral dilemma that tests Opik's values and morals.

**The Themes:**

Weave in themes that resonate with modern readers, such as the pursuit of knowledge, the dangers of imperialism, the importance of cultural exchange and understanding, and the perils of obsession. These themes would add depth and resonance to the story, making it more engaging and memorable.

Overall, "The Lost Expedition of Opik" has all the ingredients for a thrilling adventure story. By developing the characters, world-building, and plot, you'll create a compelling narrative that will keep readers on the edge of their seats. Good luck with your project!

The trace can now be viewed in the UI:

![Groq Cookbook](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/cookbook/groq_trace_decorator_cookbook.png)